## Pool Simulator!

Main loop of the game, with alternating user input. Also currently includes the new Player class, which will probably become a new file on its own. Mostly what's missing rn is the logic to figure out what player should play at each turn of the loop, which is what i felt the need of a Player class for.

In [1]:
# imports

import numpy as np
import matplotlib.pyplot as plt
from __future__ import annotations
from collections.abc import Callable
import import_ipynb
from BallClass import Ball
from StateClass import State

importing Jupyter notebook from BallClass.ipynb
importing Jupyter notebook from StateClass.ipynb


C:\Users\ayala\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nbformat\__init__.py:92: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
class Player():
    '''
    I felt the need for a Player class, to keep track of each players balls left to win,
    whether they're down to the right ball, have won or lost the game. This will be use in the main loop
    so we can just say what player did in each turn.
    '''
    name: str
    ID: int
    team: str
    balls: list[int]
    ballsLeft: int
    eight: bool
    winner: bool # will break the loop
    loser: bool # will break the loop
    
    def __init__(self, name: int, ID: int):
        self.name = name
        self.ID = ID
        self.team = "undefined"
        self.balls = []
        self. ballsLeft = len(self.balls)
        self.downToTheEight = False
        self.winner = False
        self.loser = False
    
    # find the initial list of balls this player has to pocket to win
    def balls_init(team: str):
        assert team == "stripes" or team == "solids", "Not a valid team."
        
        '''
        doesn't currently account for balls pocketed before team assignment 
        might have to take in the list of all balls from state
        '''
        
        if team == "solids":
            self.balls = [i for i in range(1,8)] #all solids
        else:
            self.balls = [i for i in range(9,16)] #all stripes
        
    # initialize the player's list of balls to pocket and team class variable    
    def set_team(team: str):
        self.team = team
        self.balls = self.balls_init(team)
        self.ballsLeft = len(self.balls)
        
    #what team is this ball?    
    def ball_whatTeam(ID: int):
        assert ID >= 0 and ID <= 15, "Not a valid ball ID."
        
        if ID >= 1 and ID <= 7:
            return "solid"
        elif ID >= 9 and ID <= 15:
            return "stripe"
        elif ID == 8:
            return "eight"
        elif ID == 0:
            return "cue"
        
    def scratch():
        '''
        idk how to implement a scratch yet
        next turn  logic goes here too, if you scratch it's not your turn again
        '''
        ...
        
    #if a player runs out of balls they either win or are now down to the eight
    def ranOutOfBalls(self):
        if self.downToTheEight:
            self.winner = True
        else:
            self.DownToTheEight = True
            self.balls.append(8)
            ''' player plays again, next turn logic'''
    
    # what happens when you pocket each type of ball
    def pocket(self, ball_ID: int):
        
        ball_team: str = self.ball_whatTeam(ball_ID)
        
        if ball_team == self.team:
            self.balls.remove(ball_ID)
            self.ballsLeft = len(self.balls)
            ''' if you score you play again, next turn logic'''
        
        elif ball_team == "eight":
            if self.DownToTheEight:
                self.winner == True
            else:
                self.loser == True #pocket the eight too early
        
        # wrong team or cue means it scratched
        else:
            self.scratch()
                

Overwriting Ball.py


In [ ]:
Player1 = Player("Jose", 1)
Player2 = Player("Elie", 2) 
runnig_state = State(Player1.name, Player2.name)

#every turn of the loop will be one turn
'''
while true:

    - figure out whose turn it is!!!!!!
    
    - take input for that player
    
    - update the state
    
    - check if player scored a pocket
    
     > if pocket:
         player.pocket()
     > if scratch
         player.scratch()

    - figure out who plays next

     
     > check if player runs out of balls

    
    - output graph

    -check if player wins or loses, break the loop

'''

## questions

I'm assuming the inputs will be a force and an angle, I can get that from the player as input, but where does it go, how will it go in and affect the state?

How do we get what balls went in from the state?


## notes

need to figure out how to implement logic for deciding who plays the next turn

need to write tests
